In [17]:
from itertools import product
from pathlib import Path
import pandas as pd
import altair as alt

# the resolve(strict=True) is just to fail fast if the directories change
hw1_src = (Path('..') / 'src' / 'hw1').resolve(strict=True)
data_gen_src_path = (hw1_src / 'DatasetGenerator.java').resolve(strict=True)
sort_src_path = (hw1_src / 'Main.java').resolve(strict=True)
data_bin_path = Path('.') / 'data.bin'
sorted_bin_path = Path('.') / 'sorted.bin'



def get_timing(t,n):
    # TODO: finish implementing the sorting algorithm and return real times
    
    #!java /path/to/datagen.java /path/to/data.bin $n
    #elapsed = !time java /path/to/java/file.java /path/to/data.bin /path/to/sorted.bin
    
    # fake timings for now
    return int(n / (t**0.9)) or 1


In [18]:
T = [2**i for i in range(7)]
N = [10**i for i in range(5, 9)]
combinations = list(product(T, N))
timings = [(t, n, get_timing(t,n)) for t,n in combinations]
df = pd.DataFrame(timings, columns=["threads", "input size", "time (seconds)"])

In [20]:
alt.Chart(df).mark_line().encode(
  x='input size',
  y='threads',
  color='time (seconds)'
)

alt.Chart(...)